# **Modelos Benchmarks**

In [28]:
# Librerias Importadas

import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.stattools import jarque_bera

In [2]:
# Cargar datos
df = pd.read_csv(r"C:\Users\TAWTOCA\OneDrive - Universidad del Norte\Documentos\Visual Files\16-21.csv")

columns_to_drop = ['CODIGO_ENTIDAD','NOMBRE_ENTIDAD', 'NOMBRE_TIPO_PATRIMONIO','NOMBRE_PATRIMONIO','TIPO_PARTICIPACION', 'CODIGO_NEGOCIO', 'TIPO_NEGOCIO', 'ANULACIONES','NUMERO_INVERSIONISTAS','NUMERO_UNIDADES_FONDO_CIERRE_OPER_DIA_T_ANTERIOR','VALOR_UNIDAD_OPERACIONES_DIA_T']

df = df.drop(columns=columns_to_drop, errors='ignore')

df.head()

,FECHA_CORTE,TIPO_ENTIDAD,NOMBRE_TIPO_ENTIDAD,SUBTIPO_NEGOCIO,NOMBRE_SUBTIPO_PATRIMONIO,PRINCIPAL_COMPARTIMENTO,RENDIMIENTOS_ABONADOS,PRECIERRE_FONDO_DIA_T,APORTES_RECIBIDOS,RETIROS_REDENCIONES,VALOR_FONDO_CIERRE_DIA_T,RENTABILIDAD_DIARIA,RENTABILIDAD_MENSUAL,RENTABILIDAD_SEMESTRAL,RENTABILIDAD_ANUAL
0,2016-01-01,5,SF-SOCIEDAD FIDUCIARIA,1,FIC DE TIPO GENERAL,1,1.225369e+08,9.303540e+11,0.0,0.0,9.303540e+11,4.925178,4.540000,3.020000,3.190000
1,2016-01-01,5,SF-SOCIEDAD FIDUCIARIA,7,FONDOS DE CAPITAL PRIVADO,1,2.678000e+02,1.370691e+11,0.0,0.0,1.370691e+11,0.000071,7.398338,47.588044,25.129215
2,2016-01-01,85,COMISIONISTAS DE BOLSA DE VALORES,1,FIC DE TIPO GENERAL,1,3.267797e+05,1.142338e+10,0.0,0.0,1.142338e+10,1.049611,-4.692300,0.631400,3.371184
3,2016-01-01,85,COMISIONISTAS DE BOLSA DE VALORES,7,FONDOS DE CAPITAL PRIVADO,2,-6.025692e+05,7.128130e+09,0.0,0.0,7.128130e+09,-3.038200,5.764400,0.000000,0.000000
4,2016-01-01,85,COMISIONISTAS DE BOLSA DE VALORES,1,FIC DE TIPO GENERAL,1,-4.057828e+04,5.055087e+08,0.0,0.0,5.055087e+08,-2.887314,145.398140,0.000000,0.000000


In [25]:
# Definir variables predictoras y respuesta
X = df[["APORTES_RECIBIDOS", "PRECIERRE_FONDO_DIA_T", "VALOR_FONDO_CIERRE_DIA_T", "TIPO_ENTIDAD"]]
y = df["RETIROS_REDENCIONES"]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
# Escalar los datos
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [58]:
# 1️⃣ MODELO DE CLASIFICACIÓN (Regresión Logística)
y_train_class = (y_train > 0).astype(int)
y_test_class = (y_test > 0).astype(int)

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train_class)

# Predicción: ¿Es 0 o mayor a 0?
y_test_class_pred = logistic_model.predict(X_test)

In [60]:
def calcular_metricas(y_real, y_pred):
    """Calcula métricas solo para y > 0 en MAPE, y para todos en RMSE y R²"""
    mape = mean_absolute_percentage_error(y_real[y_real > 0], y_pred[y_real > 0]) if np.any(y_real > 0) else np.nan
    rmse = mean_squared_error(y_real, y_pred, squared=False)
    r2 = r2_score(y_real, y_pred)
    
    # Residuales
    residuos = y_real - y_pred

    # Ljung-Box Test (Autocorrelación en residuos)
    ljung_box_pvalue = acorr_ljungbox(residuos, lags=[10], return_df=True)['lb_pvalue'].values[0]

    # Jarque-Bera Test (Normalidad en residuos)
    jarque_bera_pvalue = jarque_bera(residuos)[1]

    return {
        "MAPE": mape,
        "RMSE": rmse,
        "R²": r2,
        "Ljung-Box p-value": ljung_box_pvalue,
        "Jarque-Bera p-value": jarque_bera_pvalue
    }

In [61]:
def entrenar_modelo_regresion(X_train, X_test, y_train, y_test, metodo_regresion="ridge"):
    """Entrena un modelo de regresión, imprime R² en train/test y devuelve las predicciones"""
    
    # Filtrar datos donde y_train > 0
    X_train_reg = X_train[y_train > 0]
    y_train_reg = y_train[y_train > 0]

    # Selección del modelo de regresión
    if metodo_regresion.lower() == "ridge":
        model = Ridge(alpha=1.0)
    elif metodo_regresion.lower() == "linear":
        model = LinearRegression()
    elif metodo_regresion.lower() == "lasso":
        model = Lasso(alpha=0.1)
    else:
        raise ValueError("Método de regresión no soportado. Usa 'ridge', 'linear' o 'lasso'.")

    # Entrenar el modelo
    model.fit(X_train_reg, y_train_reg)

    # Predicciones
    y_train_pred = model.predict(X_train_reg)
    y_test_pred = model.predict(X_test)

    # Calcular R² (Accuracy)
    train_r2 = r2_score(y_train_reg, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Imprimir resultados
    print(f'Modelo: {metodo_regresion.capitalize()}')
    print(f'R² en entrenamiento: {train_r2:.4f}')
    print(f'R² en prueba: {test_r2:.4f}')
    print('-' * 40)

    return y_test_pred, model

In [62]:
def mostrar_predicciones(y_real, y_pred, num_filas=30):
    """Muestra las primeras N filas con valores reales vs. predichos"""
    df_results = pd.DataFrame({
        'Real': y_real[:num_filas].values,
        'Predicho': y_pred[:num_filas]
    })

    print(df_results)
    return df_results

## **Regresión Lineal**

In [ ]:
# Entrenar el modelo con Ridge
y_test_pred_l, modelo_linear = entrenar_modelo_regresion(X_train, X_test, y_train, metodo_regresion="linear")

In [ ]:
# Calcular métricas
metricas_l = calcular_metricas(y_test, y_test_pred_l)
print(metricas_l)

In [ ]:
# Mostrar primeras 15 predicciones
mostrar_predicciones(y_test, y_test_pred_l)

 El modelo tiene un ajuste muy bueno en ambos conjuntos, lo que indica que generaliza bien y no hay sobreajuste evidente

## **Regresión de Ridge**